In [3]:
#Understand the magic in the previus task
dates = ['cica', 'kutya', 'mérésihiba']
dates = [date.upper() for date in dates]

In [4]:
print(dates)

['CICA', 'KUTYA', 'MÉRÉSIHIBA']


# Webscraping multiple pages
https://data36.com/scrape-multiple-web-pages-beautiful-soup-tutorial/

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

## Collecting all bestseller book titles

In [2]:
#testing the method:
page = 1
while page != 6:
    url = f"https://www.bookdepository.com/bestsellers?page={page}"
    print(url)
    page = page + 1

https://www.bookdepository.com/bestsellers?page=1
https://www.bookdepository.com/bestsellers?page=2
https://www.bookdepository.com/bestsellers?page=3
https://www.bookdepository.com/bestsellers?page=4
https://www.bookdepository.com/bestsellers?page=5


In [4]:
page = 1
titles = []
while page != 35:
    
    #basic webscrape procedure
    url = f"https://www.bookdepository.com/bestsellers?page={page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    
    #get all title:
    for h3 in soup.find_all("h3", class_="title"):
        # .get_text() removes the tags like <h3>
        # strip remove whitespaces
        titles.append(h3.get_text(strip = True))
    page = page + 1

In [4]:
len(titles)

1020

In [3]:
for title in titles[:6]:
    print(title)

NameError: name 'titles' is not defined

## Get the formats 

In [6]:
#formats = soup.find_all("p", class_="format")s
# the same:
formats = soup.select("div.item-info p.format")

In [7]:
formats[:3]

[<p class="format">Paperback</p>,
 <p class="format">Paperback</p>,
 <p class="format">Paperback</p>]

In [8]:
#So it clears, like .get_text() and create a pandas series
format_series = pd.Series(formats)

In [9]:
format_series[:3]

0    [Paperback]
1    [Paperback]
2    [Paperback]
dtype: object

In [10]:
#get all formats (same method with titles):

page = 1
formats_all = []
while page != 35:
    url = f"https://www.bookdepository.com/bestsellers?page={page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    
    formats_on_page = soup.select("div.item-info p.format")
    
    #Add that element wich one is on scrape (index)
    for product_format in formats_on_page:
        formats_all.append(product_format.get_text())
        
    page = page + 1
    
formats_series = pd.Series(formats_all)
formats_series.value_counts()

Paperback              818
Hardback               157
Board book              19
Cards                   14
Game                     3
Mixed media product      2
Book                     2
Diary                    1
Spiral bound             1
Toy                      1
Novelty book             1
Leather                  1
dtype: int64

In [11]:
#Get the percentage value:
formats_series.value_counts(normalize = True)

Paperback              0.801961
Hardback               0.153922
Board book             0.018627
Cards                  0.013725
Game                   0.002941
Mixed media product    0.001961
Book                   0.001961
Diary                  0.000980
Spiral bound           0.000980
Toy                    0.000980
Novelty book           0.000980
Leather                0.000980
dtype: float64

## Getting publication dates

In [12]:
page = 1
dates = []
while page != 35:
    url = f"https://www.bookdepository.com/bestsellers?page={page}"
    response = requests.get(url)
    html = response.content
    soup = bs(html, "lxml")
    dates_on_page = soup.find_all("p", class_="published")
    
    for date in dates_on_page:
        dates.append((date.get_text()[-4:]))

    page = page + 1
    
dates_series = pd.Series(dates)
dates_series.value_counts()


2021    210
2022    182
2020    116
2018     91
2019     75
2017     53
2016     49
2015     44
2011     38
2012     26
2014     25
2013     19
2009     12
2010     11
2007      9
1997      8
2008      7
2005      7
2000      6
2002      5
2006      5
2004      4
2003      3
1995      3
1998      2
1994      2
1999      2
1991      2
1968      1
1996      1
1992      1
1982      1
dtype: int64

## Getting Prices


In [13]:
final_prices = []
prices = soup.find_all("p", class_="price")

for price in prices:
    original_price = price.find("span", class_="rrp")
    if original_price:
        current_price = str(original_price.previousSibling).strip()
        

In [14]:
current_price

'3\xa0497 Ft'

In [ ]:
#somehowe have to change the value of currency, but it's not the easyest thing

In [28]:
    url = f"https://www.bookdepository.com/bestsellers?page=1"
    headers = {"Accept-Language": "en-US,en;q=0.5"}
    response = requests.get(url, headers=headers)
    html = response.content
    soup = bs(html, "lxml")
    valuta = soup.find_all("option", selected="selected")

In [29]:
valuta 

[<option selected="selected" value="en_US">
                     English</option>,
 <option selected="selected" value="HUF">
 					Ft HUF</option>]

# Full code
## can't get the price cos of location

In [12]:
page = 1

#this will be a list of dictionary:
bestseller_books = []

#first 35 pages:
while page != 35:
    
    #with f"{}" -arg add a variable
      url = f"https://www.bookdepository.com/bestsellers?page={page}"
      response = requests.get(url)
      html = response.content
      soup = bs(html, "lxml")
    
    #Find every book in the given page
      for book in soup.find_all("div", class_="book-item"):
            
            #create a dictionary then save title, format and year:
            #this will be added to the list
            bestseller_book = {}
            bestseller_book["title"] = book.h3.get_text(strip=True)
            bestseller_book["format"] = book.find("p", class_="format").get_text()
            
            #year saving try
            try:
                  bestseller_book["year"] = book.find("p", class_="published").get_text()[-4:]
            #if there is no publ. info, we recive an attributeerror: 'NoneType' object...:
            except AttributeError:
                  bestseller_book["year"] = ""
                    
                    
            #price = book.find("p", class_="price")
            #try:
            #      original_price = price.find("span", class_="rrp")
            #except AttributeError:
            #      bestseller_book["price"] = ""
            #else:
            #      if original_price:
            #             current_price = str(original_price.previousSibling).strip()
            #            current_price = float(current_price.split("€")[0].replace(",", "."))
            #      else:
            #            current_price = float(price.get_text(strip=True).split("€")[0].replace(",", "."))
            #     bestseller_book["price"] = current_price
            
            
            bestseller_books.append(bestseller_book)
      page = page + 1

In [11]:
bestseller_book


{'title': 'The Appeal', 'format': 'Paperback', 'year': '2021'}